# Problema de asignación

> The problem instance has a number of agents and a number of tasks. Any agent can be assigned to perform any task, incurring some cost that may vary depending on the agent-task assignment. It is required to perform as many tasks as possible by assigning at most one agent to each task and at most one task to each agent, in such a way that the total cost of the assignment is minimized.

$$\begin{equation*}
	\begin{aligned}
		\min \quad & \sum_{j \in J} \sum_{i \in I} c_{ij} x_{ij}\\
		\text{st: }\quad &
		\begin{array}{c}
			\sum\limits_{j \in J} x_{ij} \leq 1 \quad \forall i \in I \\[10pt]
            \sum\limits_{i \in I} x_{ij} = 1 \quad \forall j \in J \\
            x_{ij} \in \{0, 1\}
		\end{array}
	\end{aligned}
\end{equation*}$$

In [1]:
# Dependencies
import ortools as OR
from ortools.linear_solver import pywraplp
import pandas as pd


In [2]:
def assignment_problem(costs_matrix):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return 'Error while creating Solver'

    # General data
    num_workers = len(costs_matrix)
    num_tasks = len(costs_matrix[0])

    # Decision variables
    # x[i, j] = 1 if worker i is assigned task j, 0 otherwise
    x = {}
    for i in range(num_workers):
        for j in range(num_tasks):
            x[i,j] = solver.IntVar(0, 1, '')

    # Restrictions
    # Each worker is assigned at most 1 task
    for i in range(num_workers):
        solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

    # Each task is assigned to exactly 1 worker
    for j in range(num_tasks):
        solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
    # Objective function
    objective_terms = []
    for i in range(num_workers):
        for j in range(num_tasks):
            objective_terms.append(costs_matrix[i][j] * x[i,j])

    solver.Minimize(solver.Sum(objective_terms))

    # Solve
    status = solver.Solve()
    
    def print_solution():
        if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
            print(f'Total cost = {solver.Objective().Value()}')
            print('--------------')
            for i in range(num_workers):
                for j in range(num_tasks):
                    # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                    if x[i, j].solution_value() > 0.5:
                        print(f'Worker {i} assigned to task {j}.' +
                            f' Cost: {costs_matrix[i][j]}')

            print('--------------')
            print(f'Solved in {solver.wall_time()} milliseconds')
            print(f'Solved in {solver.iterations()} iterations')
            print(f'Solved in {solver.nodes()} branch-and-bound nodes')

        else:
            print('No solution found.')

    print_solution()
    # Only works for continuous variables
    # ConstraintStatus = pd.DataFrame(columns=['Name', 'Dual'])
    # for i, constraint in enumerate(solver.constraints()):
    #     ConstraintStatus.loc[i] = [constraint.name(), constraint.dual_value()]
    # print(ConstraintStatus)


In [3]:
costs = [[90, 80, 75, 70],
        [35, 85, 55, 65],
        [125, 95, 90, 95],
        [45, 110, 95, 115]]



In [4]:
assignment_problem(costs) 

Total cost = 265.0
--------------
Worker 0 assigned to task 3. Cost: 70
Worker 1 assigned to task 2. Cost: 55
Worker 2 assigned to task 1. Cost: 95
Worker 3 assigned to task 0. Cost: 45
--------------
Solved in 31 milliseconds
Solved in 7 iterations
Solved in 1 branch-and-bound nodes
